## LND REST API example

### senario

* multi-hop payment (Alice -> Bob -> Charlie)

### setup

In [1]:
# load libraries
import json
from threading import Thread
from base64 import b64decode
from time import sleep
from client import LndClient, BtcClient
from util import p, dump, generate_blocks
import requests, codecs, json
from time import sleep
from configs import *


In [2]:

bitcoin = BtcClient(RPC_USER, RPC_PASS, BITCOIN_IP, RPC_PORT)
# initialize mainchain
bitcoin.generate(101)
p('block height = {}'.format(bitcoin.getblockcount()))

block height = 250


In [8]:
# node
alice   = LndClient("alice", LND_IP,  LND_REST_PORT)
bob     = LndClient("bob", LND_IP_BOB, LND_REST_PORT)
charlie = LndClient("charlie", LND_IP_CHARLIE, LND_REST_PORT)

### 1. fund Alice, Bob

* Alice: 0.09 btc
* Charlie: 0.08 btc

In [9]:
p('[wallet balance before funding]')
p(" alice = {}".format(alice.get('/balance/blockchain')))
p(" bob   = {}".format(  bob.get('/balance/blockchain')))

addr_a = alice.get('/newaddress', {'type': 'np2wkh'})['address']
addr_b =   bob.get('/newaddress', {'type': 'np2wkh'})['address']
bitcoin.sendtoaddress(addr_a, 0.09)
bitcoin.sendtoaddress(addr_b, 0.08)
bitcoin.generate(6)

p('[wallet balance after funding]')
p(" alice = {}".format(alice.get('/balance/blockchain')))
p(" bob   = {}".format(  bob.get('/balance/blockchain')))

[wallet balance before funding]
 alice = {'total_balance': '9000000', 'confirmed_balance': '9000000', 'unconfirmed_balance': '0', 'locked_balance': '0', 'account_balance': {'default': {'confirmed_balance': '9000000', 'unconfirmed_balance': '0'}}}
 bob   = {'total_balance': '8000000', 'confirmed_balance': '8000000', 'unconfirmed_balance': '0', 'locked_balance': '0', 'account_balance': {'default': {'confirmed_balance': '8000000', 'unconfirmed_balance': '0'}}}
[wallet balance after funding]
 alice = {'total_balance': '18000000', 'confirmed_balance': '18000000', 'unconfirmed_balance': '0', 'locked_balance': '0', 'account_balance': {'default': {'confirmed_balance': '18000000', 'unconfirmed_balance': '0'}}}
 bob   = {'total_balance': '16000000', 'confirmed_balance': '16000000', 'unconfirmed_balance': '0', 'locked_balance': '0', 'account_balance': {'default': {'confirmed_balance': '16000000', 'unconfirmed_balance': '0'}}}


### 2. connect nodes

* Alice -> Bob
* Bob   -> Charlie

In [22]:
# Alice -> Bob
pubkey_b = bob.get('/getinfo')['identity_pubkey']
host_b = f'{LND_IP_BOB}:9735'
alice.post('/peers', { 'addr': { 'pubkey': pubkey_b, 'host': host_b }, 'perm': True })

# Bob -> Charlie
pubkey_c = charlie.get('/getinfo')['identity_pubkey']
host_c = f'{LND_IP_CHARLIE}:9735'
bob.post('/peers', { 'addr': { 'pubkey': pubkey_c, 'host': host_c }, 'perm': True })

p('[identity pubkey]')
p("  bob     = {}".format(pubkey_b))
p("  charlie = {}".format(pubkey_c))

p('[peer]')
p('  alice   <-> ', end=''); dump(  alice.get('/peers'))
p('  bob     <-> ', end=''); dump(    bob.get('/peers'))
p('  charlie <-> ', end=''); dump(charlie.get('/peers'))

[identity pubkey]
  bob     = 0388ca89452064b0c7f8eb84a86c41302d59f60f304c3ae8d1dd70cd1ec6a129b1
  charlie = 024fb7f950443d01ca8c7b84ec5cea10ad0ef4e14cb4b23b8cb030aac1f337771e
[peer]
  alice   <-> {
  "peers": [
    {
      "pub_key": "0388ca89452064b0c7f8eb84a86c41302d59f60f304c3ae8d1dd70cd1ec6a129b1",
      "address": "10.21.21.20:9735",
      "bytes_sent": "391",
      "bytes_recv": "391",
      "sat_sent": "0",
      "sat_recv": "0",
      "inbound": false,
      "ping_time": "0",
      "sync_type": "ACTIVE_SYNC",
      "features": {
        "0": {
          "name": "data-loss-protect",
          "is_required": true,
          "is_known": true
        },
        "5": {
          "name": "upfront-shutdown-script",
          "is_required": false,
          "is_known": true
        },
        "7": {
          "name": "gossip-queries",
          "is_required": false,
          "is_known": true
        },
        "9": {
          "name": "tlv-onion",
          "is_required": false,
    

### 3. open the channel

In [23]:
# Alice to Bob
point_a =  alice.post('/channels', { 
    "node_pubkey_string": pubkey_b, "local_funding_amount": "7000000", "push_sat": "0" 
})

# Bob to Charlie
point_b = bob.post('/channels', { 
    "node_pubkey_string": pubkey_c, "local_funding_amount": "6000000", "push_sat": "0"
})

# open the channel
sleep(2)
bitcoin.generate(6)

# check the channel state
p('[channels: alice]'); dump(alice.get('/channels'))
p('[channels: bob]'  ); dump(  bob.get('/channels'))

[channels: alice]
{
  "channels": [
    {
      "active": true,
      "remote_pubkey": "0388ca89452064b0c7f8eb84a86c41302d59f60f304c3ae8d1dd70cd1ec6a129b1",
      "channel_point": "09778991259dfe77871fe0db757fc7831c5dd919b87ef4d6c2ce2a8b4749bf5f:1",
      "chan_id": "1014849232568321",
      "capacity": "7000000",
      "local_balance": "6999056",
      "remote_balance": "0",
      "commit_fee": "614",
      "commit_weight": "772",
      "fee_per_kw": "253",
      "unsettled_balance": "0",
      "total_satoshis_sent": "0",
      "total_satoshis_received": "0",
      "num_updates": "0",
      "pending_htlcs": [],
      "csv_delay": 841,
      "private": false,
      "initiator": true,
      "chan_status_flags": "ChanStatusDefault",
      "local_chan_reserve_sat": "70000",
      "remote_chan_reserve_sat": "70000",
      "static_remote_key": false,
      "commitment_type": "ANCHORS",
      "lifetime": "1",
      "uptime": "1",
      "close_address": "",
      "push_amount_sat": "0",
     

In [24]:
p('[channel: Alice <-> Bob]')
funding_tx_id_a = b64decode(point_a['funding_txid_bytes'].encode())[::-1].hex()
output_index_a = point_a['output_index']
p('  funding tx txid   = {}'.format(funding_tx_id_a))
p('  funding tx vout n = {}'.format(output_index_a))

p('[channel: Bob <-> Charlie]')
funding_tx_id_b  = b64decode(point_b['funding_txid_bytes'].encode())[::-1].hex()
output_index_b = point_b['output_index']
p('  funding tx txid   = {}'.format(funding_tx_id_b))
p('  funding tx vout n = {}'.format(output_index_b))

[channel: Alice <-> Bob]
  funding tx txid   = 09778991259dfe77871fe0db757fc7831c5dd919b87ef4d6c2ce2a8b4749bf5f
  funding tx vout n = 1
[channel: Bob <-> Charlie]
  funding tx txid   = 71f4ca61475453ec7929acdc4e6f1e39baada92c3e860675a7b2df53b4dd6459
  funding tx vout n = 1


### 4. create invoice

* Charlie charges Alice 40,000 satoshi

In [25]:
# add a invoice to the invoice database
invoice = charlie.post('/invoices', { "value": "40000" })

# check the invoice
invoice_info = charlie.get('/invoice/' + b64decode(invoice['r_hash'].encode()).hex())
p('[invoice]'); dump(invoice_info)

# check the payment request
payreq = charlie.get('/payreq/' + invoice['payment_request'])
p('[payment request]'); dump(payreq)

[invoice]
{
  "memo": "",
  "r_preimage": "SwZpJURyJul9PdF+3mnWoeUDE2tsGLqHmFR6TBIwlgI=",
  "r_hash": "vIaDnJjvU6+Oiy6qKxY/g4NzJBVVuGKV9TRASTXiJzs=",
  "value": "40000",
  "value_msat": "40000000",
  "settled": false,
  "creation_date": "1653232804",
  "settle_date": "0",
  "payment_request": "lnbcrt400u1p3g549ypp5hjrg88ycaaf6lr5t964zk93lswphxfq42kux9904x3qyjd0zyuasdqqcqzpgxqyz5vqsp5rdac6v694m02f7fwfg4anggsl6d0egu9qzq03uua8sty9wlh44us9qyyssq4x5sl3pqgtl2tv34590f6w8rwpxt6vwnsq3a9k9gss63qjhtkgvkpgerftugkmxn8jjt8tkx9tlm4ghu9yz5z7qppr6g4wrs2apazpgqtr4lq5",
  "description_hash": "",
  "expiry": "86400",
  "fallback_addr": "",
  "cltv_expiry": "40",
  "route_hints": [],
  "private": false,
  "add_index": "1",
  "settle_index": "0",
  "amt_paid": "0",
  "amt_paid_sat": "0",
  "amt_paid_msat": "0",
  "state": "OPEN",
  "htlcs": [],
  "features": {
    "9": {
      "name": "tlv-onion",
      "is_required": false,
      "is_known": true
    },
    "14": {
      "name": "payment-addr",
      "is_r

### 5. send the payment

* Alice pays 40,000 satoshi to Charlie
* If you have the error "unable to find a path to destination", please wait a little while and then try again.

In [26]:
# check the channel balance
p('[channel balance before paying]')
p('  alice   = {}'.format(  alice.get('/balance/channels')))
p('  charlie = {}'.format(charlie.get('/balance/channels')))

# send the payment
payment = alice.post('/channels/transactions', { 'payment_request': invoice['payment_request'] })
p('[payment]'); dump(payment)

# check the payment
# p('[payment]'); dump(alice.get('/payments'))

# wait
sleep(2)

# check the channel balance
p('[channel balance after paying]')
p('  alice   = {}'.format(  alice.get('/balance/channels')))
p('  charlie = {}'.format(charlie.get('/balance/channels')))

[channel balance before paying]
  alice   = {'balance': '6999056', 'pending_open_balance': '0', 'local_balance': {'sat': '6999056', 'msat': '6999056000'}, 'remote_balance': {'sat': '0', 'msat': '0'}, 'unsettled_local_balance': {'sat': '0', 'msat': '0'}, 'unsettled_remote_balance': {'sat': '0', 'msat': '0'}, 'pending_open_local_balance': {'sat': '0', 'msat': '0'}, 'pending_open_remote_balance': {'sat': '0', 'msat': '0'}}
  charlie = {'balance': '0', 'pending_open_balance': '0', 'local_balance': {'sat': '0', 'msat': '0'}, 'remote_balance': {'sat': '5999056', 'msat': '5999056000'}, 'unsettled_local_balance': {'sat': '0', 'msat': '0'}, 'unsettled_remote_balance': {'sat': '0', 'msat': '0'}, 'pending_open_local_balance': {'sat': '0', 'msat': '0'}, 'pending_open_remote_balance': {'sat': '0', 'msat': '0'}}
[payment]
{
  "payment_error": "",
  "payment_preimage": "SwZpJURyJul9PdF+3mnWoeUDE2tsGLqHmFR6TBIwlgI=",
  "payment_route": {
    "total_time_lock": 1041,
    "total_fees": "1",
    "total_a

### 6. close the channel

In [27]:
# check the balance
p('[channel balance]')
p(' alice   = {}'.format(  alice.get('/balance/channels')))
p(' bob     = {}'.format(    bob.get('/balance/channels')))
p(' charlie = {}'.format(charlie.get('/balance/channels')))

p('[wallet balance]')
p(' alice   = ', end=''); dump(  alice.get('/balance/blockchain')['confirmed_balance'])
p(' bob     = ', end=''); dump(    bob.get('/balance/blockchain')['confirmed_balance'])
p(' charlie = ', end=''); dump(charlie.get('/balance/blockchain')['confirmed_balance'])


p('[channel: Alice <-> Bob]')

# mine mainchain 1 block after 3 sec
Thread(target=generate_blocks, args=(bitcoin, 1, 3)).start()

# check the channel state before closing
p(' number of channels : {}'.format(len(alice.get('/channels')['channels'])))

# close the channel
res = alice.delete('/channels/' + funding_tx_id_a + '/' + str(output_index_a)).text.split("\n")[1]
closing_txid_a = json.loads(res)['result']['chan_close']['closing_txid']
p(' closing_txid = {}'.format(closing_txid_a))
sleep(5)

# check the channel state after closing
p(' number of channels : {}'.format(len(alice.get('/channels')['channels'])))


p('[channel: Bob <-> Charlie]')

# mine mainchain 1 block after 3 sec
Thread(target=generate_blocks, args=(bitcoin, 1, 3)).start()

# close the channel
res = bob.delete('/channels/' + funding_tx_id_b + '/' + str(output_index_b)).text.split("\n")[1]
closing_txid_b = json.loads(res)['result']['chan_close']['closing_txid']
p(' closing_txid = {}'.format(closing_txid_b))
sleep(5)

# check the balance
p('[channel balance]')
p(' alice   = {}'.format(  alice.get('/balance/channels')))
p(' bob     = {}'.format(    bob.get('/balance/channels')))
p(' charlie = {}'.format(charlie.get('/balance/channels')))

p('[wallet balance]')
p(' alice   = ', end=''); dump(  alice.get('/balance/blockchain')['confirmed_balance'])
p(' bob     = ', end=''); dump(    bob.get('/balance/blockchain')['confirmed_balance'])
p(' charlie = ', end=''); dump(charlie.get('/balance/blockchain')['confirmed_balance'])

[channel balance]
 alice   = {'balance': '6959054', 'pending_open_balance': '0', 'local_balance': {'sat': '6959054', 'msat': '6959054960'}, 'remote_balance': {'sat': '40001', 'msat': '40001040'}, 'unsettled_local_balance': {'sat': '0', 'msat': '0'}, 'unsettled_remote_balance': {'sat': '0', 'msat': '0'}, 'pending_open_local_balance': {'sat': '0', 'msat': '0'}, 'pending_open_remote_balance': {'sat': '0', 'msat': '0'}}
 bob     = {'balance': '5999057', 'pending_open_balance': '0', 'local_balance': {'sat': '5999057', 'msat': '5999057040'}, 'remote_balance': {'sat': '6999054', 'msat': '6999054960'}, 'unsettled_local_balance': {'sat': '0', 'msat': '0'}, 'unsettled_remote_balance': {'sat': '0', 'msat': '0'}, 'pending_open_local_balance': {'sat': '0', 'msat': '0'}, 'pending_open_remote_balance': {'sat': '0', 'msat': '0'}}
 charlie = {'balance': '40000', 'pending_open_balance': '0', 'local_balance': {'sat': '40000', 'msat': '40000000'}, 'remote_balance': {'sat': '5959056', 'msat': '5959056000'}